# **Analysis and Visualisations**

## Objectives

* Analyse how professional snooker has **internationalised since the 1980s.**
* Compute season-level **KPIs** and produce clear visuals for:
    * **International events**: Compare the number of UK and International tournaments.
    * **Players**: Analyse the number of international players per seasons and see how well they perform in each tournament
    * **Money**: Average prize money per event (UK vs Rest of World), has the prize pool become diluted?
* Explore correlations:
    * Number of International players per season ↔ Number of International Tournaments per season
    * International presence in late stages of tournament ↔ Rise in number of International Tournaments/Players
    * Rise in number of international tournaments ↔ Average prize pool per event
* Deliver static (Matplotlib/Seaborn) and interactive (Plotly) charts suitable for the final report.

## Inputs

* **Processed data from ETL (read-only in this notebook):**

  * `datasets/clean/master_cleaned.csv` (Master dataset, used for filtering and breaking down into smaller datasets)
  * `datasets/clean/triple_crown_events.csv`(Subset of the master data set only including triple crown events)
  * 


## Outputs

* Write here which files, code or artefacts you generate by the end of the notebook 

## Additional Comments

* **Definitions:**
  * **UK** = England, Scotland, Wales, Northern Ireland, and Ireland.
  * **International event** = tournament hosted outside the UK.
  * **International player** = nationality not in UK home nations.
  * **International Match** = One or more player's nationality is not in UK home nations.
  * **Late stage (QF+)** = Quarter Final, Semi Final, Final
  * **Triple Crown** = World Championship, UK Championship, Masters.

## Context

### A brief introduction to the history of snooker

Snooker is a **billiards** sport developed in the late 19th century by British Army officers stationed in India. The term *snooker* was army slang for a novice. After early variations, a standardised ruleset (including the respotted black) was adopted in **1919**.

In the 1920s, **Joe Davis**—an English professional—organised the first **World Snooker Championship (1926–27)** and then dominated the early era, winning **15 consecutive** titles before retiring unbeaten. Through these decades, snooker remained largely a club game with modest audiences.

The sport’s modern boom arrived with **colour television in the UK**. To showcase the new medium, the BBC launched the one-frame TV event ***Pot Black*** in the late 1960s, proving snooker’s TV appeal and paving the way for extensive coverage of the **World Championship**, **Masters**, and **UK Championship**—the **Triple Crown**—which the BBC still broadcasts.

#### Why early snooker was so UK-based

* **Administration and venues** were UK-centred; since **1977**, the World Championship has been staged at the **Crucible Theatre, Sheffield**, anchoring the sport’s pinnacle in Britain. ([wpbsa.com][1])
* **Broadcast economics** (notably the BBC) concentrated visibility and money in the **UK market**.
* The **player base and champions** were overwhelmingly from the **UK & Ireland**, reinforcing a domestic core.

#### Non-UK world champions

Depending on definition, there are **five (modern era)** non-UK winners:

* **Modern era / since the Crucible era**: **Cliff Thorburn** (Canada, 1980), **Neil Robertson** (Australia, 2010), **Luca Brecel** (Belgium, 2023), **Zhao Xintong** (China, 2025). ([Wikipedia][2], [Sky Sports][3])
* **Including pre-Crucible**: add **Horace Lindrum** (Australia, 1952). ([Wikipedia][4])

Notably, **Brecel in 2023** became the first champion from mainland Europe, and **Zhao Xintong in 2025** became the first Chinese (and Asian) world champion—two non-UK winners in the **last three completed championships**. ([The Guardian][5], [Sky Sports][3])

---

[1]: https://wpbsa.com/about-us/history/?utm_source=chatgpt.com "History Timeline | WPBSA | Snooker"
[2]: https://en.wikipedia.org/wiki/1980_World_Snooker_Championship?utm_source=chatgpt.com "1980 World Snooker Championship"
[3]: https://www.skysports.com/more-sports/snooker/news/12243/13349846/world-snooker-championship-2025-schedule-scores-and-results-from-the-crucible-ronnie-osullivan-judd-trump?utm_source=chatgpt.com "World Snooker Championship 2025 - Sky Sports"
[4]: https://en.wikipedia.org/wiki/World_Snooker_Championship?utm_source=chatgpt.com "World Snooker Championship"
[5]: https://www.theguardian.com/sport/live/2023/may/01/world-snooker-championship-crucible-2023-final-mark-selby-luca-brecel-live?utm_source=chatgpt.com "Luca Brecel defeats Mark Selby to win World Snooker Championship 2023 final - as it happened"


# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

# Section 1

Section 1 content

---

# Section 2

Section 2 content

---

NOTE

* You may add as many sections as you want, as long as it supports your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

---

# Push files to Repo

* In cases where you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [ ]:
import os
try:
  # create your folder here
  # os.makedirs(name='')
except Exception as e:
  print(e)
